In [4]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler


chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])
chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world-class international chef. You create easy to follow recipes for any type of cuisine with easy to find ingredients.",
        ),
        ("human", "I want to cook {cuisine} food."),
    ]
)
chef_chain = chef_prompt | chat

In [5]:
vegetarian_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it.",
        ),
        ("human", "{recipe}"),
    ]
)
vegetarian_chain = vegetarian_chef_prompt | chat

final_chain = {"recipe": chef_chain} | vegetarian_chain

final_chain.invoke(
    {
        "cuisine": "italian",
    }
)

Great choice! Let's make a classic and delicious Spaghetti Aglio e Olio. Here's the recipe:

Ingredients:
- 8 oz spaghetti
- 4 cloves of garlic, thinly sliced
- 1/4 cup extra virgin olive oil
- 1/2 tsp red pepper flakes
- Salt and pepper to taste
- Grated Parmesan cheese (optional)
- Chopped fresh parsley for garnish

Instructions:
1. Cook the spaghetti in a large pot of salted boiling water according to package instructions until al dente. Reserve 1 cup of pasta water before draining the spaghetti.

2. In a large skillet, heat the olive oil over medium heat. Add the sliced garlic and red pepper flakes. Cook, stirring frequently, until the garlic is golden and fragrant, about 2-3 minutes. Be careful not to burn the garlic.

3. Add the cooked spaghetti to the skillet along with 1/2 cup of the reserved pasta water. Toss the spaghetti in the garlic oil mixture, adding more pasta water as needed to create a light sauce that coats the pasta.

4. Season with salt and pepper to taste. If desi

AIMessage(content='For the Spaghetti Aglio e Olio recipe, here are some vegetarian alternatives and preparation tips:\n\n- **Spaghetti**: No alternative needed, as spaghetti is already vegetarian.\n\n- **Garlic**: Garlic is a crucial ingredient in this recipe. You can use the same amount of garlic as mentioned in the recipe. To prepare, peel the garlic cloves and thinly slice them as directed.\n\n- **Extra Virgin Olive Oil**: Extra virgin olive oil is perfect for this dish. Make sure to use a good quality one for the best flavor.\n\n- **Red Pepper Flakes**: Red pepper flakes add a nice kick to the dish. You can use the same amount as mentioned in the recipe.\n\n- **Salt and Pepper**: Use salt and pepper as per your taste preference.\n\n- **Grated Parmesan Cheese**: If you want to keep the dish fully vegetarian, you can use vegetarian-friendly Parmesan cheese or a similar hard cheese alternative. There are several brands that offer vegetarian versions of Parmesan cheese made without ani